<a href="https://colab.research.google.com/github/MIrfaanA/classification-predict-streamlit-template/blob/master/Copy_of_Team_6_(neater).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2021/22 Climate Change Belief Analysis Predict Solution

<img src="https://1qa2mv1tvhvk1t1uyv1mvwyx1dmd-wpengine.netdna-ssl.com/files/2014/01/Twitter-350.jpg"
     alt="Learn good habits to avoid modeling debt"
     style="float: center; padding-bottom=0.5em"
     width=700px
     height=500px/>

### Team 6 

- Floyd Skakane
- Lehlogonolo Jesica Teffo
- Muhammed Irfaan Ahmed
- Mulalo Malange
- Vuyisile Ngobeni


### Supervisor
James Combrink

### Introduction

Many companies are built around lessening one’s environmental impact or carbon footprint.They offer  products and services that are environmentally friendly and sustainable, in line with their values  and ideals.They would like to determine how people perceive climate change and whether or not they believe  it is a real threat.This would add to their market research efforts in gauging how their product/service may be received.

### Problem statement

We are tasked to create a Machine Learning model that is able to classify whether or not a person believes in climate change,
based on their novel tweet data.Providing an accurate and robust solution to this task gives companies access to a broad base
of consumer sentiment, spanning multiple demographic and geographic categories - thus increasing their insights and informing
future marketing strategies.

### Aim 
To predict an individual’s belief in climate change based on historical tweet data.

### Data description

- The collection of this data was funded by a Canada Foundation for Innovation JELF Grant to Chris Bauch, University of   Waterloo.
- The dataset aggregates tweets pertaining to climate change collected between Apr 27, 2015 and Feb 21, 2018. 
- In total, 43943 tweets were collected. 
- Each tweet is labelled as one of the following classes:

Class description:

- 2 News: the tweet links to factual news about climate change
- 1 Pro: the tweet supports the belief of man-made climate change
- 0 Neutral: the tweet neither supports nor refutes the belief of man-made climate change
- -1 Anti: the tweet does not believe in man-made climate change
- Columns:
- sentiment: Sentiment of tweet i.e 0, 1, etc
- message: Tweet body
- tweetid: Twitter unique id


<a id="cont"></a>

### Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Data Preprocessing</a>

<a href=#four>4. Exploratory Data Analysis (EDA)</a>

<a href=#five>5. Data Engineering</a>

<a href=#six>6. Modeling</a>

<a href=#seven>7. Model Performance</a>

<a href=#eight>8. Model Explanation</a>

<a href=#nine>9. Conculsion</a>

### 1. Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.stem.porter import *
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.preprocessing import StandardScaler
# ML Libraries
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
import os

# Global Parameters
stop_words = set(stopwords.words('english'))

# Setting global constants to ensure notebook results are reproducible
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 20)
sns.set(rc={'figure.figsize':(12,8)})

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### 2. Loading Data

In [ ]:
# Loading train and test data from csv files
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test_with_no_labels.csv")

### 3. Data Preprocessing

- Data preprocessing refers to the technique of cleaning and organizing raw data to make it suitable for a building and  training Machine Learning model.
- In this section unecessary data will be droped,null values will be filled with mode, the train and test datasets will be merged so that all the data preprocessing is done simultaneously on both datasets.

#### 3.1 Raw Data 

Lets have a look at the raw data to see what the data set contains, before preprocessing

In [ ]:
#Viewing the train dataset
display(df_train.head())
print(df_train.shape)
print('The train data set consists of 15819 rows and 3 columns')

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


(15819, 3)
The train data set consists of 15819 rows and 3 columns


In [ ]:
#Viewing the test dataset
display(df_test.head())
print(df_test.shape)
print('The test data set consists of 10546 rows and 2 columns')

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


(10546, 2)
The test data set consists of 10546 rows and 2 columns


The dataset consist of -----  rows and ---- variables of which -----  is the target variable.

In [ ]:
print(df_train.shape)
print('The train data set consists of 15819 rows and 3 columns')

In [ ]:
# Make a copy of the train data set for cleaning
df_clean = df_train.copy()

In [ ]:
#Empty rows in data setup
df_clean.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

### 4. Exploratory Data Analysis

In [ ]:
# Make a copy of the df so that if we make mistakes we can always refer to the original df
df_clean = df_train.copy()

### 5. Data Engineering

### 6. Modeling

### 7. Model Performance

### 8. Model Explanation

### 9. Conclusion

### Making a submission